In [21]:
import sys
import json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback
sys.path.append("../")
from src.Evaluation import plot_model_history
from sktime.datasets import load_from_tsfile_to_dataframe
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder, get_Resnet
from src.LoadData import get_all_datasets_test_train_np_arrays, get_all_datasets_info, load_numpy_array_from_ts
from src.Helpers import append_to_csv
from src.Helpers import append_to_csv, get_confusion_matrix_for_model_and_data, visualize_confusion_matrix

In [22]:
path_to_datasets = "./../datasets"

In [23]:
datasets_test_train_data = get_all_datasets_test_train_np_arrays(path_to_datasets)

In [24]:
datasets_test_train_data = {k:v for k,v in datasets_test_train_data.items() if k=='large_kitchen_appliances' }

In [25]:
models_getter = [get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder, get_Resnet]
models_names = ["MLP", "MCDCNN", "Time_CNN", "FCN", "Encoder", "Resnet"]

In [26]:
# tf.debugging.set_log_device_placement(True)
# devices = tf.config.list_logical_devices('GPU') # Use this to run training just on GPUs
devices = tf.config.list_logical_devices()
strategy = tf.distribute.MirroredStrategy(devices)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [27]:
devices

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [28]:
path_persist_results = "./training_res.csv"
if not (csv_path := Path(path_persist_results)).exists():
    csv_path.touch()
if len(csv_path.read_text()) == 0:
    # Only write if the file is empty
    columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
    append_to_csv(path_persist_results, columns)

In [29]:
epochs = 30
batch_size = 10
validation_split = 0.2

In [30]:
# with strategy.scope():
for ds_name, ds_data in tqdm(datasets_test_train_data.items(), unit='dataset'):
    print("Dataset name: ", ds_name)
    x_test, y_test = ds_data["test_data"]
    x_train, y_train = ds_data["train_data"]

    input_size = x_train.shape[1]
    output_size = len(np.unique(y_train))

    for get_model, model_name in tqdm(list(zip(models_getter, models_names)), unit='model', desc=f'Train on "{ds_name}"'):
        print("Model name: ", model_name)
        model = get_model(input_size, output_size)
        model.compile(optimizer='SGD',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[TqdmCallback(verbose=0, desc=model_name)], verbose=0)
        test_loss, test_acc = model.evaluate(x_test, y_test)
        
        confusion_matrix = get_confusion_matrix_for_model_and_data(model, x_test, y_test)
        #visualize_confusion_matrix(confusion_matrix, model_name, ds_name)

        row = [ds_name,
              model_name,
              test_loss,
              test_acc,
              json.dumps(confusion_matrix.tolist()), 
              json.dumps(history.history)]
        append_to_csv(path_persist_results, row)

  0%|          | 0/1 [00:00<?, ?dataset/s]

Dataset name:  large_kitchen_appliances


Train on "large_kitchen_appliances":   0%|          | 0/6 [00:00<?, ?model/s]

Model name:  MLP


MLP: 0epoch [00:00, ?epoch/s]

...

12/12 [==============================] - 1s 111ms/step
Model name:  Encoder


Encoder: 0epoch [00:00, ?epoch/s]

12/12 [==============================] - 5s 385ms/step
Model name:  Resnet


Resnet: 0epoch [00:00, ?epoch/s]

12/12 [==============================] - 3s 234ms/step


# Example load training results and display evaluation results

In [ ]:
df_training_res = pd.read_csv(path_persist_results)

In [ ]:
df_training_res

In [ ]:
history = df_training_res['history'][0]

In [ ]:
plot_model_history(json.loads(history), epochs=epochs)